In [1]:
import tensorflow as tf 
import keras
from datetime import datetime


In [38]:
class SimpleModule(tf.Module):
    def __init__ (self, name=None):
        super().__init__(name=name)
        self.a_variable = tf.Variable(5.0, name='train_me')
        self.non_trainable_variable=tf.Variable(5.0, trainable=False , name= "do_not_train")
    def __call__(self, x):
        return self.a_variable * x + self.non_trainable_variable

simple_module = SimpleModule(name="simple")
simple_module(tf.constant(5.0))


<tf.Tensor: shape=(), dtype=float32, numpy=30.0>

# Saving weight

In [49]:
chKp_path = "my_checkpoint"
checkpoint = tf.train.Checkpoint(model=simple_module)
checkpoint.write(chkp_path)


NameError: name 'chkp_path' is not defined

# saving function
# @tf.function

In [108]:
class MySequentiolModule(tf.Module):
    def __init__(self, name=None):
        super().__init__(name=name)
        
        self.dense_1 = Dense(in_feature=3, out_feature=3)
        self.dense_2 = Dense(in_feature=4, out_feature=6)
    
    @tf.function
    def __call__(self,x):
         x= self.dense_1(x)
         return self.dense_2(x)

# model with a gragh
my_model =  MySequentiolModule(name="the_model")


NameError: name 'Dense' is not defined

# visualize the graph by tracing it within a tensorflow summary


In [115]:
# set up logging
stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = "logs/func/%s" % stamp

new_model = MySequentialModule()

tf.summary.trace_on(graph=True)
tf.profiler.experimental.start(logdir)

z= print(new_model(tf.constant([3,5,5])))
with writer. as_default():
    tf.summary.trace_export(
    name="my_func_trace",
    step=0,
    profiler_out=logdir)


NameError: name 'MySequentialModule' is not defined

In [5]:
new_model = tf.saved_model.load("the_saved_model")

NameError: name 'tf' is not defined

# keras models and layers

In [75]:
import tensorflow as tf


In [37]:
class MyDense(tf.keras.layers.Layer):
    def __init__(self, in_feature,out_features,**kwargs):
        super().__init__(**kwargs)
        # this will soon move to the step;see below
        self.w = tf.Variable(
            tf.random.normal([in_feature, out_features]), name='w')
        self.b = tf.Variable(tf.zeros([out_features]), name='b')
    def call (self, x):
        y = tf.matmul(x, self.w)+self.b
        return tf.nn.relu(y)
simple_layer = MyDense(name="simple",in_feature=3, out_features=3)



        

In [41]:
MyDense.variables

In [43]:
# calling function allocates appropriately-size variable

In [55]:
print("model results:",simple_layer(tf.constant([[2.0,2.0,2.0],[3.0,3.0,3.0]])))

model results: tf.Tensor(
[[0. 0. 0.]
 [0. 0. 0.]], shape=(2, 3), dtype=float32)


# keras models

In [105]:
class FlexibleDense(tf.keras.layers.Layer):
  # Note the added `**kwargs`, as Keras supports many arguments
  def __init__(self, out_features, **kwargs):
    super().__init__(**kwargs)
    self.out_features = out_features

  def build(self, input_shape):  # Create the state of the layer (weights)
    self.w = tf.Variable(
      tf.random.normal([input_shape[-1], self.out_features]), name='w')
    self.b = tf.Variable(tf.zeros([self.out_features]), name='b')

  def call(self, inputs):  # Defines the computation from inputs to outputs
    return tf.matmul(inputs, self.w) + self.b

# Create the instance of the layer
flexible_dense = FlexibleDense(out_features=3)

In [117]:
# calling the allocates appropriately-sized variable:
print("model results:" ,flexible_dense(tf.constant([[2.0,2.0,2.0,],[3,3,3,]])))

model results: tf.Tensor(
[[-2.3619466  5.9696527  3.4632452]
 [-3.5429196  8.954478   5.194868 ]], shape=(2, 3), dtype=float32)


In [129]:
# calling the allocates appropriately-sized variable:
print("model results:" ,flexible_dense(tf.constant([[2.0,2.0,2.0]])))

model results: tf.Tensor([[-2.3619466  5.9696527  3.4632452]], shape=(1, 3), dtype=float32)


In [121]:
flexible_dense.variables

[]

In [103]:
try :
    print("Model results:",flexible_dense(tf.constant[[2,2,2,2]]))
except tf.errors.InvalidArgumentError as e:
    print("Faild:",e)

TypeError: 'function' object is not subscriptable

In [99]:
@keras.saving.register_keras_serializable()
class MySequentialModel(tf.keras.Model):
    def __init__(self, name=None, **kwargs):
        super().__init__(**kwargs)
        self.dense_1 = FlexibleDense(out_features=3)
        self.dense_2 = FlexibleDense(out_features=2)
    def call(self, x):
        x = self.dense_1(x)
        return self.dense_2(x)

my_sequential_model = MySequentialModel("the_model")

        

In [131]:
my_sequential_model.submodules

AttributeError: 'MySequentialModel' object has no attribute 'submodules'

In [139]:
inputs = tf.keras.Input(shape=[3,])

x = FlexibleDense(3)(inputs)
x = FlexibleDense(3)(x)

my_functional_model  =tf.keras.Model(inputs =inputs, outputs=x)

my_functional_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 3)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flexible_dense_7                │ (None, 3)              │             0 │
│ (FlexibleDense)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flexible_dense_8                │ (None, 3)              │             0 │
│ (FlexibleDense)                 │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)